<a href="https://colab.research.google.com/github/WomenInDataScience-Seattle/FortuneCookie/blob/master/FortuneCookieModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from io import StringIO

import numpy as np
import pandas as pd
import requests
url='https://raw.githubusercontent.com/WomenInDataScience-Seattle/Machine_Learning_Projects/master/FortuneCookie/training_data/data.csv'
s=requests.get(url).text

c=pd.read_csv(StringIO(s))

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from scipy.spatial.distance import cdist

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [3]:

# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, LSTM, Dropout
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import keras.utils as ku 

Using TensorFlow backend.


In [4]:
# random-word used to generate the first word in the sequence
!pip install random-word
from random_word import RandomWords

     |████████████████████████████████| 163kB 1.4MB/s eta 0:00:01


In [5]:
c.head(5)

,Fortune Cookie Quotes,Unnamed: 1
0,Be a generous friend and a fair enemy,NaN
1,Never quit!,NaN
2,"Old friends, old wines and old gold are best",NaN
3,"If your desires are not extravagant, they will...",NaN
4,Every Friend Joys in your Success,NaN


In [6]:
fortune_data = c['Fortune Cookie Quotes']

In [7]:
fortune_data.head(5)

0                Be a generous friend and a fair enemy
1                                         Never quit! 
2        Old friends, old wines and old gold are best 
3    If your desires are not extravagant, they will...
4                   Every Friend Joys in your Success 
Name: Fortune Cookie Quotes, dtype: object

In [8]:
fortune_data[1]

'Never quit! '

In [9]:
fortune_data[36]

'Let your heart make your decisions - it does not get as confused as your head. '

In [10]:
cleaned_df = fortune_data.str.lower()
cleaned_df2 = cleaned_df.str.strip()

In [11]:
dropped = cleaned_df2.dropna()

In [12]:
dropped.tail(5)

1188    your quick wits will get you out of a tough si...
1189                       your reputation is your wealth
1190                  your success will astonish everyone
1191    your talents will be recognized and suitably r...
1192    your work interests can capture the highest st...
Name: Fortune Cookie Quotes, dtype: object

In [13]:
cleaned_fortunes = dropped

In [14]:
cleaned_fortunes.head(5)

0                be a generous friend and a fair enemy
1                                          never quit!
2         old friends, old wines and old gold are best
3    if your desires are not extravagant, they will...
4                    every friend joys in your success
Name: Fortune Cookie Quotes, dtype: object

In [15]:
cleaned_fortunes[3]

'if your desires are not extravagant, they will be granted'

In [16]:
cleaned_fortunes[0]

'be a generous friend and a fair enemy'

In [17]:
corpus = cleaned_fortunes

In [18]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]

[[10, 6],
 [10, 6, 345],
 [10, 6, 345, 128],
 [10, 6, 345, 128, 8],
 [10, 6, 345, 128, 8, 6],
 [10, 6, 345, 128, 8, 6, 598],
 [10, 6, 345, 128, 8, 6, 598, 289],
 [23, 961],
 [109, 85],
 [109, 85, 109]]

In [19]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [20]:
predictors[60]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  60, 963, 221,
         8,  33], dtype=int32)

In [21]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 50, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(GRU(100, activation='relu'))
    model.add(Dropout(0.2))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 41, 50)            104350    
_________________________________________________________________
gru (GRU)                    (None, 100)               45300     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 2087)              210787    
Total params: 360,437
Trainable params: 360,437
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(predictors, label, epochs=100, verbose=5)


Instructions for updating:
Use tf.cast instead.
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100

In [23]:
# the original generate text function from https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms

def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            print(predicted)
            print(np.sum(predicted))
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()
  

In [24]:
# tweaked generate text function that uses np.random.choice to sample of the probaility distribution of the predicted word

def generate_text_prob(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_proba(token_list, verbose=0)
        random = np.random.choice(predicted.shape[1],1, p=predicted[0])
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == random:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()
  

In [25]:
token_list = tokenizer.texts_to_sequences('you')[0]
token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
predicted = model.predict_proba(token_list, verbose=0)
random = np.random.choice(predicted.shape[1],1, p=predicted[0])

print(random)
predicted[0].shape

[1157]


(2087,)

In [28]:
r = RandomWords()
random_word = 'Will'
text = generate_text_prob(random_word, 7, model, max_sequence_len)
print(text)

Will Lead The Way Of A Home Is


What we did today: 
- we changed to gru 
- we increased the word embedding length
- we increased the dropout
- we changed the activation from tanh to relu
- we randomly sampled our probaility distribution of word predictions

Next time:
- Use a pre-trained word embedding applied to our corpus
- get more data
- try training


In [29]:

print(type(model))

<class 'tensorflow.python.keras.engine.sequential.Sequential'>


In [42]:
pd.to_pickle(‘model.pkl’)

SyntaxError: invalid character in identifier (<ipython-input-42-338bae8460ba>, line 1)

In [34]:
print(model.to_json())

{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "Embedding", "config": {"name": "embedding", "trainable": true, "batch_input_shape": [null, 41], "dtype": "float32", "input_dim": 2087, "output_dim": 50, "embeddings_initializer": {"class_name": "RandomUniform", "config": {"minval": -0.05, "maxval": 0.05, "seed": null, "dtype": "float32"}}, "embeddings_regularizer": null, "activity_regularizer": null, "embeddings_constraint": null, "mask_zero": false, "input_length": 41}}, {"class_name": "GRU", "config": {"name": "gru", "trainable": true, "dtype": "float32", "return_sequences": false, "return_state": false, "go_backwards": false, "stateful": false, "unroll": false, "time_major": false, "units": 100, "activation": "relu", "recurrent_activation": "hard_sigmoid", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null, "dtype": "float32"}}, "recurrent_initializer": {"class_name": "Orthogonal", "config": {"gain": 

In [37]:
model.save('new_model.h5')

In [40]:
from keras.models import load_model

In [41]:
new_model = load_model('new_model.h5')

ValueError: Unknown initializer: GlorotUniform